In [1]:
from __future__ import print_function

import tensorflow as tf
from tensorflow.contrib import rnn
from sklearn.model_selection import train_test_split
import numpy as np

In [2]:
###load data first
#E1 data
#features = np.loadtxt("features_all.txt", delimiter=",")
#labels = np.loadtxt("labels_all.txt", delimiter=",")
#E2data
features = np.loadtxt("features_all_2.txt", delimiter=",")
labels = np.loadtxt("labels_all_2.txt", delimiter=",")
x_all = features
#y_all = labels
y_all = np.array([labels, -(labels-1)]).T 
# normalize
x_all = (x_all - x_all.min(0)) / x_all.ptp(0)
#Split the data in train & test
x_train, x_test, y_train, y_test = train_test_split(x_all, y_all)

In [3]:
# Training Parameters

# Training Parameters

learning_rate = 0.1
training_steps = 4000
batch_size = 100
display_step = 200

# Network Parameters
num_input = 61 # 
timesteps = 4 # timesteps
num_hidden = 61 # hidden layer num of features
num_classes = 2 # MNIST total classes (0-9 digits)

# tf Graph input
X = tf.placeholder("float", [None, timesteps, num_input])
Y = tf.placeholder("float", [None, num_classes])
# Define weights
weights = {
    'out': tf.Variable(tf.random_normal([num_hidden, num_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([num_classes]))
}

def RNN(x, weights, biases):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, timesteps, n_input)
    # Required shape: 'timesteps' tensors list of shape (batch_size, n_input)

    # Unstack to get a list of 'timesteps' tensors of shape (batch_size, n_input)
    x = tf.unstack(x, timesteps, 1)

    # Define a lstm cell with tensorflow
    lstm_cell = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)

    # Get lstm cell output
    outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)

    # Linear activation, using rnn inner loop last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

logits = RNN(X, weights, biases)
prediction = tf.nn.softmax(logits)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Evaluate model (with test logits, for dropout to be disabled)
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [4]:
# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)
    # total_batch = int(len(X_train)/batch_size)
    # X_batches = np.array_split(x_train, total_batch)
    # Y_batches = np.array_split(y_train, total_batch)
    
    for step in range(1, training_steps+1):
        # batch_x, batch_y = mnist.train.next_batch(batch_size)
        rand_index = np.random.choice(len(x_train), size=batch_size)
        batch_x = x_train[rand_index]
        batch_y = y_train[rand_index]
        # Reshape data to get 28 seq of 28 elements
        batch_x = batch_x.reshape((batch_size, timesteps, num_input))
        # Run optimization op (backprop)
        #print(batch_x,batch_y)
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                 Y: batch_y})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.6f}".format(acc))

    print("Optimization Finished!")

    # Calculate accuracy for 128 mnist test images
    #test_len = 128
    # test_data = mnist.test.images[:test_len].reshape((-1, timesteps, num_input))
    x_test = x_test.reshape((-1, timesteps, num_input))
    # test_label = mnist.test.labels[:test_len]
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={X: x_test, Y: y_test}))

Step 1, Minibatch Loss= 2.0211, Training Accuracy= 0.260000
Step 200, Minibatch Loss= 0.5002, Training Accuracy= 0.780000
Step 400, Minibatch Loss= 0.4318, Training Accuracy= 0.860000
Step 600, Minibatch Loss= 0.5290, Training Accuracy= 0.750000
Step 800, Minibatch Loss= 0.3069, Training Accuracy= 0.870000
Step 1000, Minibatch Loss= 0.3767, Training Accuracy= 0.840000
Step 1200, Minibatch Loss= 0.3495, Training Accuracy= 0.860000
Step 1400, Minibatch Loss= 0.3661, Training Accuracy= 0.840000
Step 1600, Minibatch Loss= 0.2077, Training Accuracy= 0.950000
Step 1800, Minibatch Loss= 0.1976, Training Accuracy= 0.970000
Step 2000, Minibatch Loss= 0.2031, Training Accuracy= 0.930000
Step 2200, Minibatch Loss= 0.1554, Training Accuracy= 0.980000
Step 2400, Minibatch Loss= 0.0766, Training Accuracy= 1.000000
Step 2600, Minibatch Loss= 0.1882, Training Accuracy= 0.930000
Step 2800, Minibatch Loss= 0.1985, Training Accuracy= 0.930000
Step 3000, Minibatch Loss= 0.0630, Training Accuracy= 0.970000